In [ ]:
!pip install openai pinecone gradio tiktoken --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 15.6 MB/s eta 0:00:00


In [ ]:
import openai
import pinecone
import os
import gradio as gr
import tiktoken
from typing import List
from tqdm.auto import tqdm

In [ ]:
openai.api_key = ""
pc = pinecone.Pinecone(api_key="", environment="")

In [ ]:
INDEX_NAME = "rag-business-qa"

if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(name=INDEX_NAME, dimension=1536, metric="cosine")
index = pc.Index(INDEX_NAME)

In [ ]:
def chunk_text(text: str, max_tokens=500) -> List[str]:
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    return ["".join(tokenizer.decode(chunk)) for chunk in chunks]

def embed_texts(texts: List[str]) -> List[List[float]]:
    client = openai.OpenAI(api_key=openai.api_key)
    response = client.embeddings.create(
        input=texts,
        model="text-embedding-3-small"
    )
    return [d.embedding for d in response.data]

def add_documents_to_pinecone(docs: List[str]):
    chunks = []
    for doc in docs:
        chunks.extend(chunk_text(doc))

    embeddings = embed_texts(chunks)
    ids = [f"doc-{i}" for i in range(len(embeddings))]
    to_upsert = list(zip(ids, embeddings, [{"text": chunk} for chunk in chunks]))
    index.upsert(vectors=to_upsert)
    print(f"✅ Uploaded {len(to_upsert)} chunks.")

In [ ]:
business_docs = [
    "Company A provides digital marketing services, including SEO, PPC, and analytics.",
    "Our customer support operates 24/7 with an average response time of 2 minutes.",
    "The enterprise plan includes unlimited users, advanced dashboards, and AI integrations.",
    # Add more business-related content as required
]

add_documents_to_pinecone(business_docs)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
def retrieve_context(query: str, k=5) -> str:
    query_embedding = embed_texts([query])[0]
    results = index.query(vector=query_embedding, top_k=k, include_metadata=True)
    contexts = [match['metadata']['text'] for match in results['matches']]
    return "\n---\n".join(contexts)

def generate_answer(query: str) -> str:
    context = retrieve_context(query)
    prompt = f"""Answer the question based on the context below:

Context:
{context}

Question: {query}

Answer:"""

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Or use "gpt-4"
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

In [ ]:
def qa_bot_interface(question):
    return generate_answer(question)

gr.Interface(fn=qa_bot_interface,
             inputs="text",
             outputs="text",
             title="📊 Business QA RAG Bot",
             description="Ask any question related to your business knowledge base").launch()